In [2]:
import numpy as np

# Collect sectional forces and moments (n_x, n_y, n_xy, m_x, m_y, m_xy)
# Shear forces are treated separately
# R = [n_x,
#      n_y,
#      n_xy,
#      m_x,
#      m_y,
#      m_xy]


n_x = 0
n_y = 0
n_xy = 0
m_x = 0
m_y = 0
m_xy = 0

R = np.array([[n_x] ,
              [n_y] ,
              [n_xy],
              [m_x] ,
              [m_y] ,
              [m_xy]])
print('R = ', R)

# Generalized strain vector

# Define the transformation matrix A
A = np.array([
    [1, 0, 0, -1, 0, 0],
    [0, 1, 0, 0, -1, 0],
    [0, 0, 1, 0, 0, -1]
])

# Define the variables (strain and curvature components)
epsilon_t = np.array([
    [0],  # ε_xm
    [0],  # ε_ym
    [0],  # ε_xym
    [0],  # κ_x
    [0],  # κ_y
    [0]   # κ_xy
])

# Define the thickness coordinate z (can be a scalar or array)
z = 1  # Example value

# Compute the strain vector ε
epsilon = A @ epsilon_t * z

print('Strain Vector (ε):')
print(epsilon)


R =  [[0]
 [0]
 [0]
 [0]
 [0]
 [0]]
Strain Vector (ε):
[[0]
 [0]
 [0]]


In [3]:
"""Experimenting with the shell section class"""

from structuralcodes.core.base import Section

class ShellSection(Section):
    def __init__(self, geometry, material, thickness):
        super().__init__(geometry, material)
        self.thickness = thickness


def compute_strain(self, epsilon_t):
        # Define the transformation matrix A
        A = np.array([
            [1, 0, 0, -1, 0, 0],
            [0, 1, 0, 0, -1, 0],
            [0, 0, 1, 0, 0, -1]
        ])

        # Define the thickness coordinate z (can be a scalar or array)
        z = self.thickness

        # Compute the strain vector ε
        epsilon = A @ epsilon_t * z

        return epsilon

def compute_stress(self, epsilon):
    # Compute the stress vector σ
    sigma = self.material.compute_stress(epsilon)

    return sigma

def compute_internal_forces(self, epsilon_t):
    # Compute the strain vector ε
    epsilon = self.compute_strain(epsilon_t)

    # Compute the stress vector σ
    sigma = self.compute_stress(epsilon)

    # Compute the internal forces vector S
    S = self.geometry.compute_internal_forces(sigma)

    return S

In [4]:
from structuralcodes.core.base import ConstitutiveLaw, Material


class ShellReinforcement:
    """Represents reinforcement in a shell section.

    Attributes:
        position_over_thickness (float): Normalised position in thickness (0 = bottom, 1 = top).
        num_bars (int): Number of bars in each bundle.
        center_distance (float): Spacing between bar bundles (mm).
        bar_diameter (float): Diameter of individual bars (mm).
        material (Material): The material of the reinforcement.
        orientation (float): Orientation angle in degrees (0° = horizontal, 90° = vertical).
    """

    def __init__(
        self,
        position_over_thickness: float,
        num_bars: int,
        center_distance: float,
        bar_diameter: float,
        material: 'Material',
        orientation: float,
    ) -> None:
        """Initialize a ShellReinforcement object.

        Arguments:
            position_over_thickness (float): Depth position as a fraction of shell thickness.
            num_bars (int): Number of bars per bundle.
            center_distance (float): Spacing between bundles (mm).
            bar_diameter (float): Bar diameter (mm).
            material (Material): Reinforcement material.
            orientation (float): Angle of reinforcement (degrees).
        """
        if not (0.0 <= position_over_thickness <= 1.0):
            raise ValueError('position_over_thickness must be between 0 and 1.')
        if num_bars <= 0:
            raise ValueError('num_bars must be positive.')
        if center_distance <= 0:
            raise ValueError('center_distance must be positive.')
        if bar_diameter <= 0:
            raise ValueError('bar_diameter must be positive.')

        self.position_over_thickness = position_over_thickness
        self.num_bars = num_bars
        self.center_distance = center_distance
        self.bar_diameter = bar_diameter
        self.material = material
        self.orientation = orientation


In [5]:
import typing as t

from structuralcodes.core.base import ConstitutiveLaw, Material

#from structuralcodes.materials.concrete import Concrete
#from structuralcodes.materials.constitutive_laws import Elastic
from structuralcodes.geometry import Geometry


class ShellGeometry(Geometry):
    """Class for representing the geometry of a shell section.

    The shell section is defined by its thickness and material properties.

    Attributes:
        thickness (float): Thickness of the shell section.
        material (Material | ConstitutiveLaw): The material model used.
        reinforcement (list[ShellReinforcement]): List of reinforcement layers.
        density (Optional[float]): Density of the shell material.
        concrete (bool): Whether the shell material is concrete.
    """

    def __init__(
        self,
        thickness: float,
        material: t.Union['Material', 'ConstitutiveLaw'],
        reinforcement: t.Optional[t.List['ShellReinforcement']] = None,
        density: t.Optional[float] = None,
        concrete: bool = False,
        name: t.Optional[str] = None,
        group_label: t.Optional[str] = None,
    ) -> None:
        """Initialize a ShellGeometry object.

        Arguments:
            thickness (float): The thickness of the shell.
            material (Union[Material, ConstitutiveLaw]): The material or
                constitutive law defining the shell's response.
            reinforcement (Optional[List[ShellReinforcement]]): Reinforcement
                layers within the shell.
            density (Optional[float]): Density of the material.
            concrete (bool): Flag to indicate if the material is concrete.
            name (Optional[str]): Name of the shell geometry.
            group_label (Optional[str]): Label for grouping several geometries.
        """
        super().__init__(name=name, group_label=group_label)

        if thickness <= 0:
            raise ValueError('Shell thickness must be a positive number.')

        if not isinstance(material, (Material, ConstitutiveLaw)):
            raise TypeError('Material must be a valid Material or ''ConstitutiveLaw object.')

        self._thickness = thickness
        self._material = material
        self._density = density if density is not None else (
            material.density if isinstance(material, Material) else None)
        self._concrete = concrete
        self._reinforcement = reinforcement if reinforcement else []

    @property
    def thickness(self) -> float:
        """Returns the thickness of the shell."""
        return self._thickness

    @property
    def material(self) -> 'ConstitutiveLaw':
        """Returns the material of the shell."""
        return self._material

    @property
    def density(self) -> float:
        """Returns the density of the shell material."""
        return self._density

    @property
    def reinforcement(self) -> t.List['ShellReinforcement']:
        """Returns the reinforcement layers within the shell."""
        return self._reinforcement

    @property
    def concrete(self) -> bool:
        """Returns True if the shell is made of concrete."""
        return self._concrete

    def add_reinforcement(self, reinforcement: "ShellReinforcement") -> None:
        """Adds a reinforcement layer to the shell."""
        self._reinforcement.append(reinforcement)

    def _repr_svg_(self) -> str:
        """Returns a simple SVG representation of the
        shell's thickness and reinforcement.
        """

In [6]:
import typing as t

from structuralcodes.core.base import Section

"""Shell section implemenetation."""
class ShellSection(Section):
    """This is the implementation of the shell section class.

    The section represents a shell element characterized by its thickness,
    material properties, and reinforcement layout.

    Attributes:
        geometry (ShellGeometry): The geometry of the shell, defined by its
            thickness and material.
        name (str): The name of the section.
        section_calculator (ShellSectionCalculator): The object responsible
            for performing calculations on the section (e.g., strain
            integration, moment-curvature analysis).
    """


    def __init__(
        self,
        geometry: 'ShellGeometry',
        name: t.Optional[str] = None,
        integrator: t.Literal['fiber'] = 'fiber',
        **kwargs,
    ) -> None:
        """Initialize a ShellSection.

        Arguments:
            geometry (ShellGeometry): The geometry of the shell section.
            name (str, optional): The name of the section.
            integrator (str): The name of the SectionIntegrator to use.
            kwargs (dict): Additional keyword arguments for the section
                calculator.

        Note:
            The ShellSection uses a ShellSectionCalculator for all
            calculations. This calculator relies on a SectionIntegrator
            for numerical integration. Any additional keyword arguments
            used when creating the ShellSection are passed to the
            ShellSectionCalculator to modify its behaviour.
        """
        if name is None:
            name = 'ShellSection'
        super().__init__(name)

        self.geometry = geometry
        self.section_calculator = ShellSectionCalculator(
            sec = self, integrator = integrator, **kwargs
        )
        self._gross_properties = None

    @property
    def gross_properties(self):
        """Return the gross properties of the shell section."""
        if self._gross_properties is None:
            self._gross_properties = (
                self.section_calculator.calculate_gross_section_properties()
            )
        return self._gross_properties

In [7]:
def get_stress_2d(
        self, eps: t.Tuple[float, float, float], nu: float
    ) -> np.ndarray:
        """Compute 2D stress state including Poisson effects.

        Arguments:
            eps (Tuple[float, float, float]): (eps_x, eps_y, gamma_xy) strain components.
            nu (float): Poisson's ratio.

        Returns:
            np.ndarray: (sigma_x, sigma_y, tau_xy) stress components.
        """
        eps_x, eps_y, gamma_xy = eps

        # Compute uniaxial stresses using the material's get_stress method
        sigma_x = self.get_stress(eps_x)
        sigma_y = self.get_stress(eps_y)

        # Compute shear stress
        G = self._E / (2 * (1 + nu))  # Shear modulus
        tau_xy = G * gamma_xy

        # Apply Poisson correction
        sigma_x_poisson = sigma_x + nu * sigma_y
        sigma_y_poisson = sigma_y + nu * sigma_x

        return np.array([sigma_x_poisson, sigma_y_poisson, tau_xy])



In [8]:
from structuralcodes.geometry import CompoundGeometry
from structuralcodes.sections.section_integrators import (
    SectionIntegrator,
    integrator_factory,
)


class ShellSectionCalculator:
    """Computes stress resultants for a shell section using FiberIntegrator."""

    def __init__(self, geometry: CompoundGeometry, integrator: t.Optional[SectionIntegrator] = None):
        """Initialize ShellSectionCalculator.

        Arguments:
            geometry (CompoundGeometry): The shell section geometry.
            integrator (FiberIntegrator, optional): The fiber integrator to use.
        """
        self.geometry = geometry
        self.integrator = integrator or SectionIntegrator()

    def integrate_strain_profile(
        self, strain_midplane: t.Tuple[float, float, float], curvature: t.Tuple[float, float, float]
    ) -> np.ndarray:
        """Compute membrane forces and bending moments using FiberIntegrator.

        Arguments:
            strain_midplane (Tuple[float, float, float]): (eps_x, eps_y, gamma_xy) at midplane.
            curvature (Tuple[float, float, float]): (kappa_x, kappa_y, kappa_xy) curvature components.

        Returns:
            np.ndarray: [n_x, n_y, n_xy, m_x, m_y, m_xy]
        """
        # Define strain state as [ε_x, ε_y, γ_xy, κ_x, κ_y, κ_xy]
        strain_profile = np.hstack((strain_midplane, curvature))

        # Use FiberIntegrator to integrate stresses over geometry
        stress_resultants, _ = self.integrator.integrate_strain_response_on_geometry(
            geo=self.geometry, strain=strain_profile, integrate="stress"
        )

        return np.array(stress_resultants)



In [9]:
class Concrete:
    """Concrete class inheriting from Elastic to add 2D stress calculation."""

    def __init__(self, E, nu):
        self._E = E
        self._nu = nu  # Store Poisson’s ratio

    def get_stress(self, eps: float) -> float:
        """Example stress-strain function (linear for now)."""
        return self._E * eps  # Hooke's law for now

    get_stress_2d = get_stress_2d  # Attach function to class

# Create a concrete material object
concrete = Concrete(E=30e9, nu=0.2)  # 30 GPa, ν = 0.2

# Compute 2D stress
strain_input = (-0.0015, 0.0005, 0.0002)
stress_output = concrete.get_stress_2d(strain_input, nu=0.2)

print('Stress components (sigma_x, sigma_y, tau_xy):', stress_output / 1e6, 'MPa')



Stress components (sigma_x, sigma_y, tau_xy): [-42.    6.    2.5] MPa
